In [1]:
import sympy as sp
import numpy as np
import math

# Step 0 : dénifir la fonction f() = (N-pq)²

In [2]:
#créer variable à partir d'un variable name en string, check le nom n'est pas déjà utilisé

#variales_name: une liste global contient des string qui sont le nom des variables, sert à conserver tout les nom des variables déjà créés
def create_variables(var_name):
    if isinstance(var_name, str) :#and var_name not in variables_name:
        try:
            #variables_name.append(var_name)
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol

In [3]:
def define_length(N):
    # taille de N en binaire
    length_N = math.ceil(math.log2(N))

    # length de p = l_1, length de q = l_2
    if N == 15 :
        l_1 = 2
        l_2 = 3
    else:
        k = (length_N) // 2 
        l_1 = k
        if length_N % 2 ==0:
            l_2 = k
        else:
            l_2 = k + 1

    return l_1,l_2

In [4]:
#p(pl_1-1,..p1,1)
#q(ql_2-1,..,q_1,1)

def set_variable(N):
    (l_1,l_2) = define_length(N)
    #p
    expr_p = 1
    for i in range(1,l_1):
        var_name = ('p'+ str(i))
        var = create_variables(var_name)
        expr_p = expr_p + 2**i * var
    #q
    expr_q = 1
    for i in range(1,l_2):
        var_name = ('q'+ str(i))
        var = create_variables(var_name)
        expr_q = expr_q + 2**i * var
    return (expr_p,expr_q)

In [5]:
def define_function(N):
    expr_pq = 1
    pq_variable = set_variable(N)
    for item in pq_variable:
        expr_pq = expr_pq * item
    return (N - expr_pq)**2

#expr = define_function(15)
#expr


# Step 1 réduction l'expression de la fonction en réduissant le power des variable 
###### Par exemple p_1**2 à p_1 tout simple car 1 * 1 = 1 & 0 * 0 = 0

In [6]:
# Step 1 réduction l'expression de la fonction en réduissant le power des variable 
# Par exemple p_1**2 à p_1 tout simple car 1 * 1 = 1 & 0 * 0 = 0

def reduct_variable_power_term(term):
    new_term = 1
    for variable,power in term.as_powers_dict().items():
        new_term *= variable
    return new_term

def reduct_variable_power_function(expr):
    expanded_expr = sp.expand(expr)
    expanded_reducted = 0
    for term in expanded_expr.args:
        new_term = reduct_variable_power_term(term)
        expanded_reducted += new_term
    return expanded_reducted

#expr_reduct_power = reduct_variable_power_function(expr)
#expr_reduct_power


# Step 2 réduire les termes qui ont une dimension plus que 2

In [7]:
# Definir une function de key pour sort
def sort_key(variable):
    name = str(variable)
    # Split le variable name aux deux parties: partie letter & parite number 
    letter = name[0]
    number = int(name[1])
    return (letter, number)


In [8]:
def rest_term(coef , rest_variable_liste):
    new_term = coef
    while(len(rest_variable_liste)!=0):
        new_term = new_term * rest_variable_liste[0]
        rest_variable_liste.pop()
    return new_term


# si #variables > 2 : diemension -1
def reduct_demension_term(coef,variables_liste):    
    global res

    # sélection les 3 premières variables
    [x_1, x_2, x_3] = variables_liste[:3]
    
    # garder le reste des variables
    rest_variables_liste= variables_liste[3:]
    new_expr = rest_term(coef , rest_variables_liste)

    # commence la réduction dimension
    tup=(x_1,x_2)
    key = next((k for k, v in dictionaire_auxiliary_variable.items() if v == tup), None)
    if key==None:
        var_name = "x" + str(res)
        x_4 = create_variables(var_name)
        dictionaire_auxiliary_variable[x_4]=(x_1,x_2)
        res += 1 
    else:
        x_4 = key 

    expr_reduct = x_4 * x_3 + 2 * (x_1 * x_2 - 2 * x_1 * x_4 - 2 * x_2 * x_4 + 3 * x_4)

    expr_reduct = new_expr * expr_reduct
    return sp.expand(expr_reduct)


In [9]:

##
def reduct_demension_function(expr):
    # stocker l'espression apres la réduction
    new_expr = 0

    # initialiser le variable
    need_reduction = False


    for term in expr.args:
        
        # get coefficant le constant et tout les variables en les mettant dans une liste
        variables_liste = sorted(term.free_symbols, key=sort_key)
        coef = term.as_coeff_mul()[0]

        # pour les termes qui ont plus de 2 variables, il faut appliquer la formule de réduction
        if len(variables_liste) > 2 :
            new_term = reduct_demension_term(coef , variables_liste) 
            new_expr +=  new_term
            if len(variables_liste) > 3 :  
                need_reduction = True
        else:
            # sinon, on ne fait rien et les met directement dans la nouvelle expression
            new_expr += term

    if need_reduction:
        return reduct_demension_function(new_expr)
    else:
        return new_expr

#reduct_demension_expr = reduct_demension_function(expr_reduct_power)
#reduct_demension_expr

# Step3 Transferer d'Qubo à Ising:
###### a * x_i = a * (s_i + 1) / 2 
###### (a * x_i * x_j) = a * (s_i + 1) * (s_j + 1) / 4

In [10]:
#créer variable à partir des variables dans modèle d'Ising

def create_variable_Ising(var_name):
    if isinstance(var_name, str) :#and var_name not in variables_name:
        try:
            symbol = sp.symbols(var_name)
            globals()[var_name] = symbol
            variables_Ising.append(symbol)
        except ValueError:
            print(f"{var_name} ValueError")
    return symbol


# définir H
def H_Ising(final_expression):

    n = len(variables)
    H = 0
    for i in range(1,n+1):
        var_name = ('s'+ str(i))
        create_variable_Ising(var_name)

    subs_relations = [(variables[i], (variables_Ising[i] + 1) / 2) for i in range(n)]
    new_expr = final_expression.subs(subs_relations)
    new_expr = sp.expand(new_expr)
    return new_expr

#h_qubo = H_qubo(reduct_demension_expr)
#h_qubo


In [11]:
def build_h(expr):
    #len de h est le nombre de variables 
    h=()
    for term in expr.args:
        if len(term.free_symbols) == 1:
            h = h + (term.as_coeff_mul()[0]/-2 ,)
    return h


def transfers(reduct_demension_expr):    
    terms = reduct_demension_expr.args 
    # Update le coef des termes avec seulement des termes avec dimension 1.
    new_expr = 0
    # Matrice contient les coef des termes avec dimension 2. 
    matrix_J = np.zeros((len(variables_Ising), len(variables_Ising)), dtype=int)
    for term in terms:
        if len(term.free_symbols) == 2:
            # 2-variables : créer la matrice J
            list_variable_term = sorted(term.free_symbols, key=sort_key)
            coef = term.as_coeff_mul()[0]#/4
            i = variables_Ising.index(list_variable_term[0])
            j = variables_Ising.index(list_variable_term[1])
            matrix_J[i][j] = coef/2
        else:
            # 1-variable : reconstruire une expression pour calculer le coef des termes
            new_expr += term
    tuple_h = build_h(new_expr)

    return tuple_h,matrix_J

#transfers(h_qubo)

sum up fonction

In [12]:
def get_h_and_J(N):
    #start
    expr = define_function(N)
    expr_reduct_power = reduct_variable_power_function(expr)
    reduct_demension_expr = reduct_demension_function(expr_reduct_power)
    ising_expr = H_Ising(reduct_demension_expr)
    return transfers(ising_expr)

### Qubo problème avec méthode heuristique

In [13]:
#import qubo modèle huristique
from pyqubo import Spin
from pyqubo import Binary
from pyqubo import solve_qubo
import neal

In [14]:
# définir formule  H for ising

def H_pyqubo(N):
    (h,J)=get_h_and_J(N)

    hamiltonian_expr = 0
    n=len(variables_Ising)

    # définir qubo spin; nombre de spin == nombre de variables
    spins = [Spin(f's{i + 1}') for i in range(n)]
    for i in range(n):
        for j in range(n):
            hamiltonian_expr -= J[i, j] * spins[i] * spins[j]
        hamiltonian_expr -= h[i] * spins[i]

    return hamiltonian_expr

In [15]:
def model_qubo(hamiltonian_expr):
    # compiler pour obtenir QUBO modèle
    model = hamiltonian_expr.compile()  
    #bqm = model.to_bqm()

    qubo, offset = model.to_qubo()
    solution = solve_qubo(qubo, num_reads=10)
    return solution

In [16]:
def model_qubo1(hamiltonian_expr):
    model = hamiltonian_expr.compile()  
    bqm = model.to_bqm()
    sa = neal.SimulatedAnnealingSampler()
    sampleset = sa.sample(bqm, num_reads=10)
    decoded_samples = model.decode_sampleset(sampleset)
    best_sample = min(decoded_samples, key=lambda x: x.energy)
    return best_sample.sample

# Call function

In [17]:
def factorization(N):

    hamiltonian_expr = H_pyqubo(N)
    #solution = model_qubo(hamiltonian_expr)
    solution = model_qubo1(hamiltonian_expr)
    print(variables)
    print(variables_Ising)
    print(solution)

    p_bits = []
    q_bits = []

    for varIsing, var in zip(variables_Ising, variables):
        if var.name.startswith('p'):
            p_bits.append(solution[varIsing.name])
        elif var.name.startswith('q'):
            q_bits.append(solution[varIsing.name])

    p_bits.append(1)
    q_bits.append(1) 

    p_bin_str = ''.join(map(str, p_bits))
    q_bin_str = ''.join(map(str, q_bits))

    p_dec = int(p_bin_str, 2)
    q_dec = int(q_bin_str, 2)


    print(f"La factorisation de {N} est {p_dec} * {q_dec}.")
    

In [19]:
variables=[]
dictionaire_auxiliary_variable={}
res=1
variables_Ising =[]
#variales_Ising: une liste global contient des string qui sont le nom des variables dans qubo, sert à compter le nombre de variables dans qubo
N=65
factorization(N)

[p1, p2, q1, q2, q3, x1, x2, x3, x4, x5, x6, x7, x8, x9, x10]
[s1, s2, s3, s4, s5, s6, s7, s8, s9, s10, s11, s12, s13, s14, s15]
{'s12': 1, 's1': 1, 's10': 0, 's2': 1, 's11': 1, 's13': 0, 's14': 0, 's15': 0, 's3': 1, 's4': 1, 's5': 1, 's6': 0, 's7': 0, 's8': 0, 's9': 1}
La factorisation de 65 est 7 * 15.
